# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [330]:
# Imports at the top
import json
import urllib2
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import datetime
%matplotlib inline

## Exercise 1: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 1.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

In [331]:
data = BeautifulSoup(urllib2.urlopen('http://www.imdb.com/chart/top'), "lxml")

In [332]:
len(movie_id)

250

In [333]:
tables = data.find_all('table')
table = tables[0]

movie_id = []
for idx, tr in enumerate(table.find_all('tr')):
    print tr
    if idx > 0:
        for idx1, a in enumerate(tr.find_all('a')):
            if idx1 ==0:
                movie_id.append(a['href'].split('/')[2])


<tr>
<th></th>
<th>Rank &amp; Title</th>
<th>IMDb Rating</th>
<th>Your Rating</th>
<th></th>
</tr>
<tr>
<td class="posterColumn">
<span data-value="1" name="rk"></span>
<span data-value="9.214631248099929" name="ir"></span>
<span data-value="7.791552E11" name="us"></span>
<span data-value="1726094" name="nv"></span>
<span data-value="-1.7853687519000712" name="ur"></span>
<a href="/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=2398042102&amp;pf_rd_r=1VBETXWEG1F3DB7F2M58&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=top&amp;ref_=chttp_tt_1"> <img height="67" src="https://images-na.ssl-images-amazon.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_UY67_CR0,0,45,67_AL_.jpg" width="45"/>
</a> </td>
<td class="titleColumn">
      1.
      <a href="/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&amp;pf_rd_p=2398042102&amp;pf_rd_r=1VBETXWEG1F3DB7F2M58&amp;pf_rd_s=center-1&amp;pf_rd_t=15506&amp;pf_rd_i=top&amp;ref_=chttp_tt_1" title="Frank Darabont (dir.), Tim Robbins, Mor

## 1.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
- Check the documentation of omdbapi.com to learn how to request movie data by id
- Define a function that returns a python object with all the information for a given id
- Iterate on all the IDs and store the results in a list of such objects
- Create a Pandas Dataframe from the list

In [334]:
info = pd.DataFrame()
for i in movie_id:
    soup = BeautifulSoup(urllib2.urlopen('http://www.omdbapi.com/?i='+ i), "lxml")
    parsed_json = json.loads(soup.text)
    info = pd.concat((info, (pd.DataFrame([parsed_json]))), axis = 0)

info.index = (range(len(info)))

## 1.c Get gross data

The OMDB API is great, but it does not provide information about Gross Revenue of the movie. We'll revert back to scraping for this.

- Write a function that retrieves the gross revenue from the entry page at imdb.com
- The function should handle the exception of when the page doesn't report gross revenue
- Retrieve the gross revenue for each movie and store it in a separate dataframe

In [335]:
base = 'http://www.imdb.com'
urls = []
for idx, row in enumerate(table.find_all('tr')):
    if idx > 0:
        urls.append(base + row.find('a').get('href'))

In [336]:
page = BeautifulSoup(urllib2.urlopen(urls[0]), "lxml")
tables = gross.find_all('div', attrs = {"class": "article", "id": "titleDetails" })

In [337]:
def FindGross(url):
    page = BeautifulSoup(urllib2.urlopen(url), "lxml")
    tables = page.find_all('div', attrs = {"class": "article", "id": "titleDetails" })
    gross = 0.0
    for idx, div in enumerate(tables[0].find_all('div', attrs = {"class": "txt-block"})):
            if 'Gross' in div.text:
                gross = float((div.text.split('$')[1]).split(' ')[0].replace(',', ''))
    return gross

Mov_gross = []
for url in urls:
    Mov_gross += [FindGross(url)]

In [338]:
grossdb = pd.DataFrame(Mov_gross)

## 1.d Data munging

- Now that you have movie information and gross revenue information, let's clean the two datasets.
- Check if there are null values. Be careful they may appear to be valid strings.
- Convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Merge the data from the two datasets into a single one

In [339]:
info['Released'] = info['Released'].apply(lambda x: datetime.datetime.strptime(x, '%d %b %Y') if x!= 'N/A' else np.nan)
info['Runtime'] = info['Runtime'].apply(lambda x: x.split(' ')[0])
info['imdbVotes'] = info['imdbVotes'].apply(lambda x: int(x.replace(",", "")))
info['imdbRating'] = info['imdbRating'].apply(lambda x: float(x))


In [340]:
grossdb.columns = ['Gross']
df = pd.concat((info, grossdb), axis= 1)

## 1.e Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to modify the `token_pattern` in the `CountVectorizer`.

In [341]:
infobackup = info

In [342]:
info = infobackup
info.columns.values

array([u'Actors', u'Awards', u'Country', u'Director', u'Genre',
       u'Language', u'Metascore', u'Plot', u'Poster', u'Rated',
       u'Released', u'Response', u'Runtime', u'Title', u'Type', u'Writer',
       u'Year', u'imdbID', u'imdbRating', u'imdbVotes'], dtype=object)

In [343]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 1000,
                             ngram_range=(1,1),
                             stop_words='english',
                             binary=True,
                             token_pattern='(?u)\\b\\w\\w+-?\\w+\\b'
                            )

In [344]:
xgen =  vectorizer.fit_transform(info.Genre)
genre_dummies = pd.DataFrame(xgen.todense(), columns = vectorizer.get_feature_names())

In [2]:
vectorizer = CountVectorizer(binary=True,
                             max_features=1000,
                             tokenizer=lambda x: x.split(', ')
                            )

NameError: name 'CountVectorizer' is not defined

In [3]:
xgen =  vectorizer.fit_transform(info.Actors)
actor_dummies = pd.DataFrame(xgen.todense(), columns = vectorizer.get_feature_names())

NameError: name 'vectorizer' is not defined

In [346]:
df = pd.concat((df, actor_dummies, genre_adummies), axis= 1)

## Exercise 2: Showing Our Data

We did all that hard work. Let's show it to the world! First save your .csv file.

```
df.to_csv('imdb.csv', sep='\t', encoding='utf-8')
```

In [347]:
df.to_csv('imdb.csv', sep='\t', encoding='utf-8')

## 2.a Add model

Add a new model to the models.py file in your website folder. It should contain each movie's meta data such as rank, summary, genre, and rating. Here is an example of a movies model.

```
class TopMovies(models.Model):
    name = models.CharField(max_length=200)
    rank = models.IntegerField(default=-1)
    created = models.DateTimeField(auto_now_add=True, blank=True)
```

## 2.b Add Url
Add the following path to your url.py file.

```
url(r'^imdb/top/$', views.imdb, name='imdb')
```

## 2.c Create View

Create your view. Remember you need to name it imdb! Your view should contain the following logic:

- If your imdb database has no values in it
    - Open up your imdb.csv file and populate the model
- Retrieve model and show the data on your webpage

## 2.d Create Template

Link a new template to your view. Your template must do the following things:

- Use a table, rows, and columns to organize your data
- Have a different background color besides white
- Show your image poster links as images

## 2.e Submit Url

Paste your new url below

In [349]:
# df = pd.read_csv('imdb.csv', sep='\t', encoding='utf-8')
# df['Actors']

# model_instances = []
# for index , record in df.iterrows():
#     print index
#     print record['Actors']
#     model_instances += topmovies(record['Actors'],
#                                  record['Titles'])
                                 
                                
        


# # model_instances = [topmovies(actors=df['Actors'], field_2=record['field_2'],) for record in df]

# MyModel.objects.bulk_create(model_instances)